In [1]:
import gdsfactory as gf
from shapely.geometry.polygon import Polygon
from shapely import affinity
import numpy as np

from gdsfactory.read import cell_from_yaml_template

# from faker import Faker
# from functools import partial

gf.clear_cache()

2025-03-09 17:13:31.485 | WARNING  | kfactory.kcell:show:8770 - Could not connect to klive server
2025-03-09 17:13:32.136 | WARNING  | kfactory.kcell:show:8770 - Could not connect to klive server


In [2]:
class LargeChips(gf.Component):

    def __init__(self, name, *args, **kwargs):
        super().__init__(name, *args, **kwargs)
        CHIP_X = 9200  # The length of the chip in microns  
        CHIP_Y = 9200  # The width of the chip in microns 
        STEP = 200  # The spacing between the markers in microns                     
        ROW_NUM = 10  # The number of markers in each row
        COL_NUM = 10  # The number of markers in each column
        TEXT_SIZE = 10  # The size of the text in microns
        BOUNDARY = 1  # The width of the boundary in microns
        MARKER_SIZE = 20  # The size of the marker in microns
        self.row_length = CHIP_X
        self.col_length = CHIP_Y
        self.step = STEP
        self.row_num = ROW_NUM
        self.col_num = COL_NUM
        self.text_size = TEXT_SIZE
        self.boundary = BOUNDARY
        self.marker_size = MARKER_SIZE
        self.layer = (8, 0)
        self.layer_screen_marker = (1, 0)
        self.marker_dict = {}
        def add_marker(x, y, layer=(9, 0)):
            self.__create_cross1__(f"{y/1000}", x/1000, x, y, 3, 25, 8, layer=layer)
            self.__create_cross1__(f"{-y/1000}", x/1000, x, -y, 3, 25, 8, layer=layer)
            self.__create_cross1__(f"{y/1000}", -x/1000, -x, y, 3, 25, 8, layer=layer)
            self.__create_cross1__(f"{-y/1000}", -x/1000, -x, -y, 3, 25, 8, layer=layer)
        add_marker(2400, 2400)
        self.create_marker()
        self.create_note("20240920")
        self.create_outline(width=BOUNDARY, mx=0.0)
    
    def __create_align_cross__(self, x, y, layer=None, cross=True):
        """
        The squar marker for EBL alignment
        Args:
            x (_type_): Location of the marker in x
            y (_type_): Location of the marker in y
            layer (_type_, optional): Layers. Defaults to None.
        """
        if layer is None:
            layer = self.layer
        size = 20
        marker = self << gf.components.rectangle(size=[size, size], layer=layer)
        marker.dmove((x-10, y-10))
        
        cross_length = 390
        gap = 100
        
        if cross:
            rect1 = self << gf.components.rectangle(size=[cross_length, size/2], layer=layer)
            rect1.dxmin = marker.dxmax + gap
            rect1.dymin = marker.dymin + size/4
            
            rect2 = self << gf.components.rectangle(size=[cross_length, size/2], layer=layer)
            rect2.dxmax = marker.dxmin - gap
            rect2.dymin = marker.dymin + size/4
            
            rect3 = self << gf.components.rectangle(size=[size/2, cross_length], layer=layer)
            rect3.dxmin = marker.dxmin + size/4
            rect3.dymax = marker.dymax - gap
            
            rect4 = self << gf.components.rectangle(size=[size/2, cross_length], layer=layer)
            rect4.dxmin = marker.dxmin + size/4
            rect4.dymin = marker.dymin + gap
    
    def __create_locate__(self, text, num, x, y, text_size, layer=None):
        if layer is None:
            layer = self.layer
        T1 = self << gf.components.text(text=f"{text},{num}", size=text_size, justify="right",layer=layer)
        T1.dxmin = x
        T1.dymax = y
        
    def __create_cross1__(self, text, num, x, y, cross_width, cross_length, text_size, layer=None):
        if layer is None:
            layer = self.layer
        L1 = self << gf.components.L(cross_width, size=(cross_length, cross_length), layer=layer)
        L2 = self << gf.components.L(cross_width, size=(cross_length, cross_length), layer=layer)
        T1 = self << gf.components.text(text=f"{text}", size=text_size, justify="right",layer=layer)
        N1 = self << gf.components.text(text=f"{num}", size=text_size, justify="left",layer=layer)
        L2.drotate(180,(0, 0))
        L1.dmove((x+cross_width/2, y+cross_width/2))
        L2.dmove((x-cross_width/2, y-cross_width/2))
        T1.dmove((L1.dxmax, L1.dymax-text_size))
        N1.dmove((L2.dxmin-text_size/10, L2.dymin))
        
        
    def create_marker(self, remove_markers=[]):
        row_length, col_length = self.row_length, self.col_length
        # Define the characters for the markers
        characters = [chr(i) for i in range(65, 65 + self.row_num)]  # ASCII 65 = 'A'
        
        # Define the spacing between markers
        dx, dy = self.step, self.step  # 5mm x 5mm assuming units in microns
        self.marker_dx = dx
        self.marker_dy = dy
        marker_dict = {}
        rect = gf.components.rectangle(size=[self.marker_size, self.marker_size], layer=self.layer)
        rect_arr = self.add_ref(rect, spacing=(dx, dy), columns=self.col_num, rows=self.row_num)
        rect_arr.dxmin = - rect_arr.dxsize/2
        rect_arr.dymin = - rect_arr.dysize/2
        
        
        def add_ref_arr(rect):
            defect = 2  # 没数字标记的marker的缺少数  
            rect_arr2 = self.add_ref(rect, spacing=(dx, dy), columns=1, rows=self.row_num-2*defect)
            rect_arr2.dxmin = rect_arr.dxmin - 1*dx
            rect_arr2.dymin = rect_arr.dymin + defect*dy
            
            rect_arr3 = self.add_ref(rect, spacing=(dx, dy), columns=1, rows=self.row_num-2*defect)
            rect_arr3.dxmax = rect_arr.dxmax + 1*dx
            rect_arr3.dymin = rect_arr.dymin + defect*dy
            
            rect_arr4 = self.add_ref(rect, spacing=(dx, dy), columns=self.col_num-2*defect, rows=1)
            rect_arr4.dxmin = rect_arr.dxmin + defect*dx
            rect_arr4.dymin = rect_arr.dymin - 1*dy
            
            rect_arr5 = self.add_ref(rect, spacing=(dx, dy), columns=self.col_num-2*defect, rows=1)
            rect_arr5.dxmin = rect_arr.dxmin + defect*dx
            rect_arr5.dymax = rect_arr.dymax + 1*dy
        
        # add_ref_arr(rect)
        
        def gen_cross(layer, marker_loc=2000, cross=True):
            self.__create_align_cross__(marker_loc, marker_loc, layer, cross=cross)
            self.__create_align_cross__(marker_loc, -marker_loc, layer, cross=cross)
            self.__create_align_cross__(-marker_loc, marker_loc, layer, cross=cross)
            self.__create_align_cross__(-marker_loc, -marker_loc, layer, cross=cross)
        gen_cross(layer=self.layer)
        gen_cross(layer=self.layer, marker_loc=2200, cross=False)
        gen_cross(layer=self.layer, marker_loc=1800, cross=False)
        gen_cross(layer=self.layer, marker_loc=1600, cross=False)

        # Loop through each row and column
        for i, char in enumerate(characters):
            for j in range(0, self.col_num):
                marker = self.__create_locate__(j-int(self.col_num/2)+1, i-int(self.row_num/2)+1, j*dx+self.marker_size+rect_arr.dxmin, i*dy-self.marker_size+rect_arr.dymin, self.text_size)
                index_i = i-int(self.row_num/2)+1
                index_j = j-int(self.col_num/2)+1
                locate_x = j*dx+rect_arr.dxmin
                locate_y = i*dy+rect_arr.dymin
                marker_dict |= {f"{index_j}, {index_i}" : [locate_x+self.marker_size/2, locate_y+self.marker_size/2]}  # +10 means the location of the center
                if (index_i, index_j) not in remove_markers:
                    marker = self.__create_locate__(j-int(self.col_num/2)+1, i-int(self.row_num/2)+1, j*dx+self.marker_size+rect_arr.dxmin, i*dy-self.marker_size+rect_arr.dymin, self.text_size, layer=self.layer_screen_marker)
                    rect = self << gf.components.rectangle(size=[self.marker_size, self.marker_size], layer=self.layer_screen_marker)
                    rect.dxmin, rect.dymin = locate_x, locate_y
        self.marker_dict = marker_dict
        
    def get_position(self, i, j, xshift=0, yshift=0):
        try:
            x, y = self.marker_dict[f"{i}, {j}"]
            return x + xshift, y + yshift
        except:
            print(f"Error: the markers do not create!")
            return None
        
            
    def create_outline(self, width, mx=0, my=0):  # 5mm = 5000 micrometers
        dxmin, dymin = self.dxmin, self.dymin
        dxmax, dymax = self.dxmax, self.dymax
        xgap = self.row_length - (dxmax - dxmin)
        ygap = self.col_length - (dymax - dymin)
        # Create the outer square outline of the marker array
        xshift = dxmin - xgap/2
        yshift = dymin - ygap/2
        # outer_square = self.add_polygon([(0, 0), (0, self.col_length), (self.row_length, self.col_length), (self.row_length, 0)], layer=self.layer_screen_marker)
        c = gf.Component()
        outer_square = c << gf.components.rectangle(size=[self.row_length, self.col_length], layer=self.layer_screen_marker)
        outer_square.dxmin = xshift
        outer_square.dymin = yshift
        
        # Create the inner square
        inner_row_length = self.row_length - 2 * width
        inner_col_length = self.col_length - 2 * width
        # inner_square = self.add_polygon([(width, width), (width, inner_col_length + width), (inner_row_length + width, inner_col_length + width), (inner_row_length + width, width)], layer=self.layer_screen_marker)
        inner_square = c << gf.components.rectangle(size=[inner_row_length, inner_col_length], layer=self.layer_screen_marker)
        inner_square.dxmin = xshift + width
        inner_square.dymin = yshift + width

        # Subtract the inner square from the outer square to get the outline
        square_outline_shape = self << gf.boolean(outer_square, inner_square, operation='xor')
        
        # square_outline_shape_move = affinity.translate(square_outline_shape, xshift+mx, yshift+my)
        # self.add_polygon(square_outline_shape_move, layer=self.layer)
        # self.add_polygon(square_outline_shape_move, layer=self.layer_screen_marker)
        
    def center_of_marker(self, chr1, chr2, chr3, chr4):
        x1, y1 = self.marker_dict[chr1]
        x2, y2 = self.marker_dict[chr2]
        x3, y3 = self.marker_dict[chr3]
        x4, y4 = self.marker_dict[chr4]
        x = (x1 + x2 + x3 + x4) / 4
        y = (y1 + y2 + y3 + y4) / 4
        return x, y
    
    def create_note(self, text):
        note = self << gf.components.text(text=text, position = (0, 0), layer = self.layer, size = 50)
        note.dymin = self.dymax - note.dysize
        note.dxmin = - (note.dxmin + note.dxmax)/2
        note2 = self << gf.components.text(text=text, position = (0, 0), layer = self.layer_screen_marker, size = 50)
        note2.dymin = note.dymin
        note2.dxmin = note.dxmin

In [3]:
gf.clear_cache()
chip = LargeChips("chip8")

In [4]:
chip.show()

In [5]:
class LargeChipsSuperconductor(LargeChips):
    def __init__(self, name, debug, *args, **kwargs):
        super().__init__(name, *args, **kwargs)
        self.debug = debug
        self.small_electrode_layer = (2, 0)
        self.big_electrode_layer = (3, 0)
        self.small_electrode_length = 100
        self.small_electrode_width = 2
        # self.create_electrodes()
        self.create_pads_0()
        self.measure_width1 = 25
        self.measure_width2 = 2
        self.sourece_width1 = 50
        self.sourece_width2 = 2
        self.hole_layer = (5, 0)
        self.hole_layer2 = (6, 0)
        self.create_pads_1(self.measure_width1, pad_size2=self.sourece_width1)
        # self.create_tapers_2(25, 60, 20, self.big_electrode_layer)
        self.create_tapers_0(self.measure_width2, self.measure_width1, 40, width1_2=self.sourece_width2, widht2_2=self.sourece_width1)
        self.create_straights_pre(4, 3)
        self.create_straights_0()
        self.connect_tapers2_tapers3()
        self.connect_straights()
        self.create_holes()
        self.create_holes_2()
        self.create_holes_3(-218, 128)
        self.create_holes_3(215, 128)

    
    def __debug_text__(self, text, x, y):
        if self.debug:
            add_text = self << gf.components.text(text, size=6, layer=(100, 0))
            add_text.dxmin = x
            add_text.dymin = y

    def create_electrode(self, x, y, length, width, theta, layer):
        electrode = self << gf.components.straight(length=length, width=width, layer=layer)
        electrode.drotate(theta)
        electrode.dxmin = x
        electrode.dymin = y
        return electrode

    def create_electrodes(self):
        gap = 4
        self.small_electrodes = []
        for y in np.linspace(80, 80+3*(gap+self.small_electrode_width), 4):
            small_electrode = self.create_electrode(50, y, self.small_electrode_length, self.small_electrode_width, 0, layer=self.small_electrode_layer)
            self.small_electrodes.append(small_electrode)

    def print_ports_info(self, pad, pad_group_name, pad_idx):
        self.__debug_text__(f"{pad_group_name}[{pad_idx}]", pad.dxmin, pad.dymin-20)
        for port in pad.ports:
            self.__debug_text__(f"{port.name}", port.x/1000, port.y/1000)

    def create_pads_0(self):
        self.pads_0 = []
        xlocate = 2500
        ylocate = -2200
        ygap = 500
        pad_size = 500
        y_arr = [ylocate+pad_size+ygap, ylocate+2*(pad_size+ygap), ylocate+3*(pad_size+ygap)]
        for x in [-xlocate, xlocate-pad_size]:
            for y in y_arr:
                pad = self << gf.components.pad(size=[pad_size, pad_size], layer=self.big_electrode_layer)
                pad.dxmin = x
                pad.dymin = y
                self.pads_0.append(pad)
                self.print_ports_info(pad, "pad_0", len(self.pads_0)-1)


    def create_pads_1(self, pad_size, pad_size2):
        self.pads_1 = []
        def create_pad1(x, y, rotate=0, pad_size=pad_size):
            pad = self << gf.components.pad(size=[pad_size, pad_size], layer=self.big_electrode_layer)
            pad.drotate(rotate)
            pad.dxmin = x
            pad.dymin = y
            self.pads_1.append(pad)
            self.print_ports_info(pad, "pad_1", len(self.pads_1)-1)
        yshift = 80
        x, y = self.get_position(-4, 4, yshift=yshift)
        create_pad1(x, y)

        x, y = self.get_position(-4, 3, yshift=yshift)
        create_pad1(x, y)

        x, y = self.get_position(5, 4, yshift=yshift)
        create_pad1(x, y)

        x, y = self.get_position(5, 3, yshift=yshift)
        create_pad1(x, y)

        x, y = self.get_position(5, 2, yshift=yshift)
        create_pad1(x, y, pad_size=pad_size2)

        x, y = self.get_position(-4, 2, yshift=yshift)
        create_pad1(x, y, pad_size=pad_size2)

    def create_tapers_0(self, width1, width2, length, width1_2, widht2_2):
        self.tapers_0 = []

        def create_taper0(rotate, x, y, width1=width1, width2=width2):
            # taper_c = gf.components.taper(length=length, width1=width1, width2=width2, layer=layer)
            taper_c = gf.components.taper_electrical(length=length, width1=width1, width2=width2, with_two_ports=True, layer=self.big_electrode_layer, port_names=('e1', 'e2'), port_types=('electrical', 'electrical'), with_bbox=True)
            taper = self << taper_c
            taper.drotate(rotate)
            taper.dxmin = x
            taper.dymin = y
            self.tapers_0.append(taper)
            # self.print_ports_info(taper, "tapers_0", len(self.tapers_0)-1)

        x, y = self.get_position(-1, 2, xshift=50, yshift=-length/2)
        create_taper0(90, x, y)
        create_taper0(90, x+90, y)

        x, y = self.get_position(1, 2, xshift=50, yshift=-length/2)
        create_taper0(90, x, y)
        create_taper0(90, x+90, y)

        x, y = self.get_position(2, 2, xshift=80, yshift=-length/2)
        create_taper0(90, x, y, width1=width1_2, width2=widht2_2)

        x, y = self.get_position(-2, 2, xshift=80, yshift=-length/2)
        create_taper0(90, x, y, width1=width1_2, width2=widht2_2)

        # x, y = self.get_position(-1, 2, xshift=50, yshift=-length/2)
        # create_taper0(90, x+80, y)
        # x, y = self.get_position(0, 2, xshift=50, yshift=-length/2)
        # create_taper0(90, x, y)

        # x, y = self.get_position(1, 2, xshift=50, yshift=-length/2)
        # create_taper0(90, x-120, y)
        # create_taper0(90, x, y)

        # x, y = self.get_position(2, 2, xshift=80, yshift=-length/2)
        # create_taper0(90, x-20, y, width1=width1_2, width2=widht2_2)

        # x, y = self.get_position(-2, 2, xshift=80, yshift=-length/2)
        # create_taper0(90, x+20, y, width1=width1_2, width2=widht2_2)

    def create_straights_pre(self, length, yshift):
        self.straights_pre = []
        def create_straight(width, ref, length=length):
            x = (ref.dxmin + ref.dxmax) / 2  - width / 2
            y = ref.dymin + yshift
            straight = self << gf.components.straight(width=width, length=length, port_types=("electrical", "electrical"), port_names=("e1", "e2"), layer=self.small_electrode_layer)
            straight.drotate(90)
            straight.dxmin = x
            straight.dymax = y
            self.straights_pre.append(straight)
            self.print_ports_info(straight, "straight_pre", len(self.straights_pre)-1)

        for i in [0, 1, 2, 3]:
            create_straight(self.measure_width2, self.tapers_0[i])

        for i in [4, 5]:
            create_straight(self.sourece_width2, self.tapers_0[i])
        
        

    def create_straights(self, measured_width, soureced_width, length, gap1, gap2, x, y, group_name):
        straight_arr = []
        def create_straight(width, length, x, y):
            straight = self << gf.components.straight(width=width, length=length, port_types=("electrical", "electrical"), port_names=("e1", "e2"), layer=self.small_electrode_layer)
            straight.drotate(90)
            straight.dxmin = x
            straight.dymin = y
            straight_arr.append(straight)
            self.print_ports_info(straight, group_name, len(straight_arr)-1)
        total_x = soureced_width * 2 + measured_width * 4 + gap1 * 2 + gap2 * 3
        locate_xmin = x - total_x / 2
        locate_ymin = y - length / 2
        create_straight(soureced_width, length,  locate_xmin, locate_ymin)

        create_straight(measured_width, length,  locate_xmin+soureced_width+gap2, locate_ymin)
        create_straight(measured_width, length,  locate_xmin+soureced_width+1*measured_width+gap2+gap1, locate_ymin)
        create_straight(measured_width, length,  locate_xmin+soureced_width+2*measured_width+2*gap2+gap1, locate_ymin)
        create_straight(measured_width, length,  locate_xmin+soureced_width+3*measured_width+2*gap2+2*gap1, locate_ymin)

        create_straight(soureced_width, length,  locate_xmin+soureced_width+4*measured_width+3*gap2+2*gap1, locate_ymin)

        return straight_arr
    
    def create_straights_0(self):
        x1, y1 = self.get_position(0, 0)
        x2, y2 = self.get_position(1, 1)
        x = (x1 + x2) / 2
        y = (y1 + y2) / 2 
        length = 60
        gap1 = 40 + length
        yshift = 110
        self.straights_0 = self.create_straights(self.measure_width2, self.sourece_width2, length, 5, 4, x, y+yshift, "straights_0")
        self.straights_1 = self.create_straights(self.measure_width2, self.sourece_width2, length, 4, 4, x, y+yshift-gap1, "straights_1")
        self.straights_2 = self.create_straights(self.measure_width2, self.sourece_width2, length, 3, 3, x, y+yshift-2*gap1, "straights_2")

    

    def connect_tapers2_tapers3(self):
    
        def route1(ref1, ref2, ref3, p1, p2_1, p2_2, p3, start_straight_length1=400, start_straight_length2=400):
            width1 = ref2.dymax-ref2.dymin
            routes = gf.routing.route_single_electrical(self, ref1.ports[p1], ref2.ports[p2_1], width=width1, start_straight_length=start_straight_length1, layer=self.big_electrode_layer)
            routes = gf.routing.route_single_electrical(self, ref2.ports[p2_2], ref3.ports[p3], width=width1, start_straight_length=start_straight_length2, layer=self.big_electrode_layer)
        
        route1(self.pads_0[2], self.pads_1[0], self.tapers_0[1], "e3", "e1", "e3", "e2")
        route1(self.pads_0[1], self.pads_1[1], self.tapers_0[0], "e3", "e1", "e3", "e2")
        route1(self.pads_0[5], self.pads_1[2], self.tapers_0[2], "e1", "e3", "e1", "e2")
        route1(self.pads_0[4], self.pads_1[3], self.tapers_0[3], "e1", "e3", "e1", "e2")
        route1(self.pads_0[0], self.pads_1[5], self.tapers_0[5], "e3", "e1", "e3", "e2", start_straight_length1=600)
        route1(self.pads_0[3], self.pads_1[4], self.tapers_0[4], "e1", "e3", "e1", "e2", start_straight_length1=600)

    def connect_straights(self):
    
        def route2(ref1, ref2, p1, p2, start_straight_length=0):
            width1 = ref1.dxmax-ref1.dxmin
            routes = gf.routing.route_single_electrical(self, ref1.ports[p1], ref2.ports[p2], width=width1, start_straight_length=start_straight_length, layer=self.small_electrode_layer)
        
        gap1 = 45
        route2(self.straights_pre[5], self.straights_0[0], "e1", "e2", start_straight_length=3*gap1)
        route2(self.straights_pre[4], self.straights_0[5], "e1", "e2", start_straight_length=3*gap1)
        route2(self.straights_pre[0], self.straights_0[1], "e1", "e2", start_straight_length=2*gap1)
        route2(self.straights_pre[1], self.straights_0[2], "e1", "e2", start_straight_length=gap1)
        route2(self.straights_pre[2], self.straights_0[3], "e1", "e2", start_straight_length=gap1)
        route2(self.straights_pre[3], self.straights_0[4], "e1", "e2", start_straight_length=2*gap1)

        gap2 = 10
        for i, start_length_idx in enumerate([3, 2, 1, 1, 2, 3]):
            start_length = start_length_idx * gap2
            route2(self.straights_0[i], self.straights_1[i], "e1", "e2", start_straight_length=start_length)
            route2(self.straights_1[i], self.straights_2[i], "e1", "e2", start_straight_length=start_length)
        
    def create_holes(self):

        def create_between(ref1, ref2, gap=0.01):
            width = ref2.dxmin - ref1.dxmax
            length = ref1.dymax - ref1.dymin
            hole = self << gf.components.rectangle(size=(width-2*gap, length), layer=self.hole_layer)
            hole.dxmin = ref1.dxmax + gap
            hole.dymin = ref1.dymin
            return hole
        
        self.hole1 = create_between(self.straights_0[3], self.straights_0[4], gap=1.5)
        self.hole1 = create_between(self.straights_1[3], self.straights_1[4], gap=1.5)
        self.hole1 = create_between(self.straights_2[3], self.straights_2[4], gap=0.01)
    
    def create_holes_2(self):
        radius = 2.5
        hole = gf.components.circle(radius=radius, layer=self.hole_layer)
        self.hole_array1 = self << gf.components.array(hole, columns=8, rows=9, spacing=(15, 15))
        self.hole_array1.dxmin = self.straights_pre[0].dxmin - 2.5 + 1 - 7.5
        self.hole_array1.dymax = self.straights_pre[0].dymax - 9
        
        self.hole_array2 = self << gf.components.array(hole, columns=8, rows=9, spacing=(15, 15))
        self.hole_array2.dxmax = self.straights_pre[3].dxmax + 2.5 - 1 + 7.5
        self.hole_array2.dymax = self.straights_pre[3].dymax - 9
        
    def create_holes_3(self, x0, y0):
        r1 = 0.5/2
        r2 = 2/2
        r3 = 0.1/2
        d1 = 6 + r1 + r2
        d2 = 2.5 + r2 + r3
        rowx = 27
        rowy = 3
        hole1_1 = gf.components.circle(radius=r1, layer=self.hole_layer2)
        holearray1_1 = self << gf.components.array(hole1_1, columns=2*rowx, rows=rowy, spacing=(d1, np.sqrt(3)*d1))
        holearray1_1.dx = x0
        holearray1_1.dy = y0
        
        holearray1_2 = self << gf.components.array(hole1_1, columns=rowx, rows=rowy, spacing=(2*d1, np.sqrt(3)*d1))
        holearray1_2.dx = x0 + d1
        holearray1_2.dy = y0 - np.sqrt(3)*d1/2
        
        hole2 = gf.components.circle(radius=r2, layer=self.hole_layer2)
        holearray2 = self << gf.components.array(hole2, columns=rowx, rows=rowy, spacing=(2*d1, np.sqrt(3)*d1))
        holearray2.dx = x0
        holearray2.dy = y0 - np.sqrt(3)*d1/2
        
        hole3 = gf.components.circle(radius=r3, layer=self.hole_layer2)
        holearray3_1 = self << gf.components.array(hole3, columns=rowx, rows=rowy, spacing=(2*d1, np.sqrt(3)*d1))
        holearray3_1.dx = x0
        holearray3_1.dy = y0 - np.sqrt(3)*d1/2 + d2
        
        holearray3_2 = self << gf.components.array(hole3, columns=rowx, rows=rowy, spacing=(2*d1, np.sqrt(3)*d1))
        holearray3_2.dx = x0
        holearray3_2.dy = y0 - np.sqrt(3)*d1/2 - d2
        
        holearray3_3 = self << gf.components.array(hole3, columns=rowx, rows=rowy, spacing=(2*d1, np.sqrt(3)*d1))
        holearray3_3.dx = x0 + d2 / 2 * np.sqrt(3)
        holearray3_3.dy = y0 - np.sqrt(3)*d1/2 + d2 / 2
        
        holearray3_4 = self << gf.components.array(hole3, columns=rowx, rows=rowy, spacing=(2*d1, np.sqrt(3)*d1))
        holearray3_4.dx = x0 + d2 / 2 * np.sqrt(3)
        holearray3_4.dy = y0 - np.sqrt(3)*d1/2 - d2 / 2
        
        holearray3_5 = self << gf.components.array(hole3, columns=rowx, rows=rowy, spacing=(2*d1, np.sqrt(3)*d1))
        holearray3_5.dx = x0 - d2 / 2 * np.sqrt(3)
        holearray3_5.dy = y0 - np.sqrt(3)*d1/2 + d2 / 2
        
        holearray3_6 = self << gf.components.array(hole3, columns=rowx, rows=rowy, spacing=(2*d1, np.sqrt(3)*d1))
        holearray3_6.dx = x0 - d2 / 2 * np.sqrt(3)
        holearray3_6.dy = y0 - np.sqrt(3)*d1/2 - d2 / 2
        
#['strip', 'rib', 'rib_bbox', 'rib2', 'nitride', 'strip_rib_tip', 'strip_nitride_tip', 'slot', 'rib_with_trenches', 'l_with_trenches', 'metal1', 'metal2', 'metal3', 'metal_routing', 'heater_metal', 'npp', 'pin', 'pn', 'pn_with_trenches', 'pn_with_trenches_asymmetric', 'l_wg_doped_with_trenches', 'strip_heater_metal_undercut', 'strip_heater_metal', 'strip_heater_doped', 'rib_heater_doped', 'rib_heater_doped_via_stack', 'pn_ge_detector_si_contacts']
gf.clear_cache()
chip = LargeChipsSuperconductor("chip8", debug=False)
chip.show()